## GPU Programming with Numba and CuPy
### Estimating Pi with Monte Carlo Simulation

First taking a look at the hardware we have available to work with:

In [1]:
!nvidia-smi

Wed Mar 29 15:02:37 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Serially, we can solve this problem in `numpy` like so (generating 100m coordinates in a unit square and identifying whether they fall in a unit circle or not):

In [2]:
# NumPy Pi Estimation with Monte Carlo Simulation
import numpy as np
import time

t0 = time.time()

n_runs = 10 ** 8

# Simulate Random Coordinates in Unit Square:
ran = np.random.uniform(low=-1, high=1, size=(2, n_runs))

# Identify Random Coordinates that fall within Unit Circle and count them
result = ran[0] ** 2 + ran[1] ** 2 <= 1
n_in_circle = np.sum(result)

# Estimate Pi
print("Pi Estimate: ", 4 * n_in_circle / n_runs)

print("Time Elapsed: ", time.time() - t0)

Pi Estimate:  3.141506
Time Elapsed:  2.7714076042175293


### Vectorization on GPUs with `numba`

In [3]:
from numba import vectorize, cuda
from numba.core.errors import NumbaPerformanceWarning
import warnings
warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

n_runs = 10**8
ran = np.random.uniform(low=-1, high =1, size=(2, n_runs)).astype(np.float32)
x, y = ran[0], ran[1]

@vectorize(['i4(f4, f4)'], target='cuda')
def in_circle(x, y):
  '''
  Vectorized function takes in x, y coordinates (float32, float32) within an
  array and returns a boolean indication of whether these values are (1) or
  are not (0) in the unit circle (cast as int32). 
  
  All computation is done on the GPU.
  '''
  in_circle = x**2 + y**2 <= 1
  return in_circle

@cuda.reduce
def gpu_sum(a, b):
  '''
  Sums values in an array together on the GPU, using numba's built-in
  `reduce` algorithm.
  '''
  return a + b

First, we might try to only perform the mapping operation on the GPU (passing the x and y arrays over to the GPU, computing whether the random coordinates are in the unit circle or not, and then sending values back to the CPU to sum and estimate pi):

In [4]:
%%timeit
result = in_circle(x, y) # send numpy arrays x & y to GPU and perform in_circle computation on GPU
4 * np.sum(result) / n_runs # send resulting array back to CPU to compute pi using `numpy`

343 ms ± 16.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Can also perform the summation on the GPU...

In [5]:
%%timeit
result = in_circle(x, y)
4 * gpu_sum(result) / n_runs # perform sum on GPU this time and then only two scalar ops on CPU

286 ms ± 23.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


And store intermediate results on the GPU (`result` is currently being sent back to CPU):

In [7]:
%%timeit
# Slight improvement by keeping intermediate result on GPU, but not to the
# degree of custom CUDA kernel.
in_circle_dev = cuda.device_array(shape=(n_runs,), dtype=np.float32) # allocate spot in memory on GPU for output array to live
in_circle(x, y, out=in_circle_dev) # write results of `in_circle` out to spot in memory allocated to `in_circle_dev`
4 * gpu_sum(in_circle_dev) / n_runs

206 ms ± 31.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
